In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import os

In [2]:
# Load data
data_fake = pd.read_csv("C:\\Users\\HP\\Desktop\\jupyter projects\\Fake.csv")
data_true = pd.read_csv("C:\\Users\\HP\\Desktop\\jupyter projects\\True.csv")

print(os.getcwd())


C:\Users\HP


In [3]:
# Display the first few rows of each dataset
data_fake.head()
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
# Insert class column as the target feature
data_fake["class"] = 0
data_true["class"] = 1


In [5]:
# Remove the last 10 rows for manual testing
data_fake_manual_testing = data_fake.tail(10)
data_fake.drop(data_fake.tail(10).index, inplace=True)

data_true_manual_testing = data_true.tail(10)
data_true.drop(data_true.tail(10).index, inplace=True)

In [6]:
# Add class labels to manual testing DataFrames
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\HP\AppData\Local\Temp\ipykernel_6288\614466769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_6288\614466769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [7]:
# View manual testing data
data_fake_manual_testing.head(10)
data_true_manual_testing.head(10)


,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [8]:
# Merge true and fake DataFrames
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [9]:
# Drop columns not required for further processing
data = data_merge.drop(['title', 'subject', 'date'], axis=1)


In [10]:
# Check for missing values
data.isnull().sum()


text     0
class    0
dtype: int64

In [11]:
# Shuffle the data
data = data.sample(frac=1, random_state=1)  # Added random_state for reproducibility


In [12]:
# Reset index after shuffling and drop the old index column
data.reset_index(drop=True, inplace=True)


In [13]:
# Text preprocessing function
def wordopt(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub('\[.*?\]', '', text)  # Remove text inside square brackets
    text = re.sub("\\W", " ", text)  # Remove non-word characters
    text = re.sub("https?://\S+|www\.S+", " ", text)  # Remove URLs
    text = re.sub('<.*?>', " ", text)  # Remove HTML tags
    text = re.sub('[%s]' % re.escape(string.punctuation), " ", text)  # Remove punctuation
    text = re.sub('\n', '', text)  # Remove newlines
    text = re.sub('\w*\d\w*', " ", text)  # Remove words with digits
    return text


In [14]:
# Apply text preprocessing
data['text'] = data['text'].apply(wordopt)


In [15]:
# Split data into features and target
x = data['text']
y = data['class']

In [16]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [19]:
# Initialize and fit the vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [20]:
# Initialize and train the Logistic Regression model
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [21]:
# Predict and evaluate Logistic Regression model
y_pred = LR.predict(xv_test)
print("Logistic Regression Accuracy on test set:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Logistic Regression Accuracy on test set: 0.9849376114081997
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5868
           1       0.98      0.98      0.98      5352

    accuracy                           0.98     11220
   macro avg       0.98      0.98      0.98     11220
weighted avg       0.98      0.98      0.98     11220



In [23]:
# Initialize and train the Decision Tree model
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(xv_train, y_train)


DecisionTreeClassifier()

In [24]:
# Predict and evaluate Decision Tree model
y_pred = dt.predict(xv_test)
print("Decision Tree Accuracy on test set:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Decision Tree Accuracy on test set: 0.9949197860962566
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5868
           1       0.99      0.99      0.99      5352

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [25]:
# Define function to output label
def output_label(n):
    return "Fake News" if n == 0 else "Not A Fake News"


In [27]:
# Function for manual testing
def manual_testing(news):
    # Create a DataFrame for the new test news
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    
    # Apply text preprocessing
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    # Vectorize the new test news
    new_xv_test = vectorization.transform(new_def_test["text"])
    
    # Predict with Logistic Regression and Decision Tree
    pred_LR = LR.predict(new_xv_test)
    pred_DT = dt.predict(new_xv_test)
    
    # Print predictions
    print("\n\nLR Prediction: {} \nDT Prediction: {}".format(
        output_label(pred_LR[0]), 
        output_label(pred_DT[0])
    ))

In [28]:
# Example usage:
manual_testing("This is a test news article.")



LR Prediction: Fake News 
DT Prediction: Fake News
